In [292]:
# TODO

# 1. computerTotalFantasyPoints()

In [1]:
import gazpacho as gp
import pandas as pd
from gazpacho import Soup, get

import playerposition as pp

In [2]:
# CBS Data
positionInput = input(f"Input one of: F, C, W, D, G")

position = positionInput.upper()

baseUrl = f"https://www.cbssports.com/fantasy/hockey/stats"
url = f"{baseUrl}/{position}/2021/restofseason/projections"

print(f"URL: {url}")

html = get(url)

Input one of: F, C, W, D, G f


URL: https://www.cbssports.com/fantasy/hockey/stats/F/2021/restofseason/projections


In [3]:
# League Scoring Settings

pointValues = {
    "G": 1.0,
    "A": 0.5,
    "PTS": 0.5,
    "SOG": 0.2,
    "STP": 0.5,
    "DEF": 0.5,
    "W": 3.0,
    "SV": 0.2,
    "GA": -0.5,
    "OTL": 1.0,
}

In [8]:
# Invoke this cell if you'd like to change the scoring values of your category


def is_float(check_input):
    if "." in check_input:
        split_number = check_input.split(".")
        if (
            len(split_number) == 2
            and split_number[0].isdigit()
            and split_number[1].isdigit()
        ):
            return True
    return False


def changeScoringValues():
    print(f"Please enter all values as floats (e.g. 1.0)")

    # Goals
    goalVal = input("Value of Goals")
    if is_float(goalVal):
        try:
            pointValues["G"] = float(goalVal)
            print(f"Updated value of G to {goalVal}")
        except:
            raise ValueError(f"Invalid input. Value not updated")
            pass

    # Assists
    assistVal = input("Value of Assists")
    if is_float(assistVal):
        try:
            pointValues["A"] = float(assistVal)
            print(f"Updated value of A to {assistVal}")
        except:
            raise ValueError(f"Invalid input. Value not updated")
            pass

    # Points ######
    pointVal = input("Value of Points")
    if is_float(pointVal):
        try:
            pointValues["PTS"] = float(pointVal)
            print(f"Updated value of PTS to {pointVal}")
        except:
            raise ValueError(f"Invalid input. Value not updated")
            pass

    # Shots on Goal
    sogVal = input("Value of Shots on Goal")
    if is_float(sogVal):
        try:
            pointValues["SOG"] = float(sogVal)
            print(f"Updated value of SOG to {sogVal}")
        except:
            raise ValueError(f"Invalid input. Value not updated")
            pass

    # Special Teams Points
    stpVal = input("Value of STP")
    if is_float(stpVal):
        try:
            pointValues["STP"] = float(stpVal)
            print(f"Updated value of STP to {stpVal}")
        except:
            raise ValueError(f"Invalid input. Value not updated")
            pass

    # Defenceman Points
    defVal = input("Value of DEF")
    if is_float(defVal):
        try:
            pointValues["DEF"] = float(defVal)
            print(f"Updated value of DEF to {defVal}")
        except:
            raise ValueError(f"Invalid input. Value not updated")
            pass

    # Wins
    winVal = input("Value of Wins")
    if is_float(winVal):
        try:
            pointValues["W"] = float(winVal)
            print(f"Updated value of W to {winVal}")
        except:
            raise ValueError(f"Invalid input. Value not updated")
            pass

    # Saves
    saveVal = input("Value of Saves")
    if is_float(saveVal):
        try:
            pointValues["SV"] = float(saveVal)
            print(f"Updated value of SV to {saveVal}")
        except:
            raise ValueError(f"Invalid input. Value not updated")
            pass
    5.0
    # Goals Against
    gaVal = input("Value of Goals Against (as positive value)")
    if is_float(gaVal):
        try:
            pointValues["GA"] = float(gaVal) * -1
            print(f"Updated value of GA to -{gaVal}")
        except:
            raise ValueError(f"Invalid input. Value not updated")
            pass

    # OT Loss
    otlVal = input("Value of OT Losses")
    if is_float(otlVal):
        try:
            pointValues["OTL"] = float(otlVal)
            print(f"Updated value of OTL to {otlVal}")
        except:
            raise ValueError(f"Invalid input. Value not updated")
            pass


# changeScoringValues()
print(pointValues)

Please enter all values as floats (e.g. 1.0)


Value of Goals 4
Value of Assists 5,88
Value of Points 4.5


Updated value of PTS to 4.5


Value of Shots on Goal 3.5


Updated value of SOG to 3.5


...

Updated value of SV to 3.4


Value of Goals Against (as positive value) 5.666666


Updated value of GA to -5.666666


Value of OT Losses 0.8


Updated value of OTL to 0.8
{'G': 4.5, 'A': 5.6, 'PTS': 4.5, 'SOG': 3.5, 'STP': 8.7, 'DEF': 4.7, 'W': 9.0, 'SV': 3.4, 'GA': -5.666666, 'OTL': 0.8}


In [ ]:
# Hockey-Reference Advanced Stats

ASData = pd.read_csv(f"./data/NHLAdvancedStats_Players2122.csv")

# Parsing Advanced Stats Data
advancedDF = pd.DataFrame(ASData)
cols = [3, 19, 20, 21, 22, 23]
advancedDF = advancedDF[advancedDF.columns[cols]]
print(advancedDF)

In [ ]:
# Parsing of HTML and extracting 'TableBase-bodyTr' tag
soup = Soup(html)
rows = soup.find("tr", {"class": "TableBase-bodyTr"})
headers = soup.find("tr", {"class": "TableBase-headTr"})

In [ ]:
# Data to Table (Players)


def parse_header(header, pos):
    player = header.find("a")[0].text.upper().strip()
    position = "POS"
    team = "TEAM"
    
    # Skaters
    if pos != "G":
        goalsScored = header.find("a")[4].text.upper()
        assists = header.find("a")[5].text.upper()
        points = header.find("a")[6].text.upper()
        shotsOnGoal = header.find("a")[15].text.upper()
        return player, team, position, goals, assists, points, shotsOnGoal
    # Goalies
    else:
        wins = header.find("a")[5].text.upper()
        shutouts = header.find("a")[7].text.upper()
        goalsAgainst = header.find("a")[8].text.upper()
        shotsAgainst = header.find("a")[10].text.upper()
        saves = header.find("a")[11].text.upper()
        return player, team, position, wins, shutouts, goalsAgainst, shotsAgainst, saves


def parse_row(row, pos):
    # currPlayer is its own class "CellPlayerName--long" in 'TableBase-bodyTr'
    # Find name, position, team for currPlayer
    currPlayer = row.find("span", {"class": "CellPlayerName--long"})
    name = currPlayer.find("a").text
    position = currPlayer.find("span", {"class": "CellPlayerName-position"}).text
    team = currPlayer.find("span", {"class": "CellPlayerName-team"}).text

    # Skaters
    if pos != "G":
        goalsScored = int(row.find("td", {"class": "TableBase-bodyTd"})[4].text)
        assists = int(row.find("td", {"class": "TableBase-bodyTd"})[5].text)
        points = int(row.find("td", {"class": "TableBase-bodyTd"})[6].text)
        shotsOnGoal = int(row.find("td", {"class": "TableBase-bodyTd"})[15].text)
        return name, team, position, goals, assists, points, shotsOnGoal
    # Goalies
    else:
        wins = row.find("td", {"class": "TableBase-bodyTd"})[5].text.strip()
        shutouts = row.find("td", {"class": "TableBase-bodyTd"})[7].text.strip()
        goalsAgainst = int(row.find("td", {"class": "TableBase-bodyTd"})[8].text)
        saves = int(row.find("td", {"class": "TableBase-bodyTd"})[10].text)
        shotsAgainst = int(row.find("td", {"class": "TableBase-bodyTd"})[11].text)

        return name, team, position, wins, shutouts, goalsAgainst, shotsAgainst, saves


playerInfoDF = pd.DataFrame(
    [parse_row(row, position) for row in rows], columns=parse_header(headers, position)
)
# print(playerInfoDF.columns)
# print(advancedDF.columns)
# print(playerInfoDF.columns.intersection(advancedDF.columns))


if position != "G":
    result = pd.merge(playerInfoDF, advancedDF, on="PLAYER")
else:
    result = playerInfoDF
print(result)